# Jupyter notebook

This jupyter notebook aims to illustrate a simple workflow with REHO.

In [1]:
from reho.model.reho import *
from reho.plotting import plotting

In [6]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import plotly.io as pio
pio.templates.default = "plotly"

In [3]:
# Set building parameters
reader = QBuildingsReader()
reader.establish_connection('Geneva')
qbuildings_data = reader.read_db(71, egid=['1009515'])

# Select clustering options for weather data
cluster = {'Location': 'Geneva', 'Attributes': ['T', 'I', 'W'], 'Periods': 10, 'PeriodDuration': 24}

# Set scenario
scenario = dict()
scenario['Objective'] = 'TOTEX'
scenario['EMOO'] = {}
scenario['specific'] = []
scenario['name'] = 'totex'
scenario['exclude_units'] = ['ThermalSolar', 'NG_Cogeneration']
scenario['enforce_units'] = []

# Initialize available units and grids
grids = infrastructure.initialize_grids()
units = infrastructure.initialize_units(scenario, grids)

# Set method options
method = {'building-scale': True}

# Run optimization
reho = REHO(qbuildings_data=qbuildings_data, units=units, grids=grids, cluster=cluster, scenario=scenario, method=method, solver="highs")
reho.single_optimization()

Connected to database
	host: 128.179.39.7
	port: 4443
	database: QBuildings-Geneva
	username: public_user
The weather data have been loaded from the PVGIS database for a location with coordinates {'latitude': 46.172, 'longitude': 6.094, 'elevation': 403.0}.
Applying algorithm for 10 clusters
N_k optimal: 10
The data have been computed and saved in /Users/lepour/Documents/IPESE/UrbanSystems/REHO/scripts/debug/data/clustering.
PANEL ORIENTATION: azimuth  270 , tilt  90
Limiting angle design 0
80 patches can NOT be seen
65 patches can be seen
INITIATION, Iter:0 Pareto_ID: 0
MASTER INITIATION, Iter:0
HiGHS 1.6.0: optimal solution; objective -1477.308338
0 simplex iterations
0 barrier iterations
 
         TOTEX
0 -1477.308338
Empty DataFrame
Columns: []
Index: []
LAST MASTER ITERATION, Iter:1 Pareto_ID: 0
HiGHS 1.6.0: optimal solution; objective -1477.308338
0 simplex iterations
0 barrier iterations
 
         TOTEX
0 -1477.308338
INITIATE HOUSE: Building1
HiGHS 1.6.0: optimal solution; ob

In [7]:
results = reho.results

# Performance : Economical, Environmental, and Combined
plotting.plot_performance(results, plot='costs', indexed_on='Scn_ID', title="Economical performance", filename="figures/performance_costs").show()
plotting.plot_performance(results, plot='gwp', indexed_on='Scn_ID', title="Environmental performance", filename="figures/performance_gwp").show()
plotting.plot_performance(results, plot='combined', indexed_on='Scn_ID', title="Combined performance (Economical + Environmental)", filename="figures/performance_combined").show()

# Costs and Revenues
plotting.plot_expenses(results, plot='costs', indexed_on='Scn_ID', title="Costs and Revenues").show()

# Sankey diagram
plotting.plot_sankey(results['totex'][0], label='EN_long', color='ColorPastel', title="Sankey diagram").show()

# Eud-use demands
plotting.plot_sunburst_eud(results, label='EN_long', title="End-use demands").show()

# Energy profiles
units_to_plot = ['ElectricalHeater', 'HeatPump', 'PV', 'NG_Boiler']
plotting.plot_profiles(results['totex'][0], units_to_plot, label='EN_long', color='ColorPastel', resolution='weekly', title="Energy profiles with a weekly moving average").show()